In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [17]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the API URL
api_url = 'https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise'

# Fetch the data from the API
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    covid_data = response.json()
    
    # Extract state-wise data from the JSON response
    statewise_data = covid_data['data']['statewise']

    # Prepare lists to hold the extracted data
    states = []
    cases = []
    deaths = []
    recovered = []
    active = []

    # Loop through the state data
    for state in statewise_data:
        states.append(state['state'])
        cases.append(state['confirmed'])
        deaths.append(state['deaths'])
        recovered.append(state['recovered'])
        active.append(state['active'])

    # Create a DataFrame for the processed data
    df = pd.DataFrame({
        'State': states,
        'Cases': cases,
        'Deaths': deaths,
        'Recovered': recovered,
        'Active': active
    })

    # Calculate Cure Rate and Death Rate
    df['Cure Rate (%)'] = (df['Recovered'] / df['Cases']) * 100
    df['Death Rate (%)'] = (df['Deaths'] / df['Cases']) * 100

    # Allow dynamic selection of top N affected states
    top_n = 10  # Change this value to dynamically select the number of top states

    # Sort by the number of cases to get the top N affected states
    top_states = df.sort_values(by='Cases', ascending=False).head(top_n)

    # Create a bar chart for Cure Rate and Death Rate using Plotly
    bar_fig = px.bar(
        top_states,
        x='State',
        y=['Cure Rate (%)', 'Death Rate (%)'],
        title=f'Top {top_n} States in India: COVID-19 Cure and Death Rates',
        labels={'value': 'Percentage', 'variable': 'Rate'},
        barmode='group',
        color_discrete_map={'Cure Rate (%)': 'green', 'Death Rate (%)': 'red'}  # Customizing colors
    )

    # Customize the layout
    bar_fig.update_layout(
        xaxis_title='State',
        yaxis_title='Percentage (%)',
        legend_title='Rate Type',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode='x unified'
    )

    # Create a pie chart to show the distribution of cases among top N states
    pie_fig = px.pie(
        top_states,
        names='State',
        values='Cases',
        title=f'Distribution of COVID-19 Cases in Top {top_n} States',
        color_discrete_sequence=px.colors.qualitative.Pastel
    )

    # Customize the pie chart
    pie_fig.update_traces(
        textposition='inside',
        textinfo='percent+label'
    )

    # Create a histogram to show the distribution of Cure and Death Rates
    hist_fig = make_subplots(specs=[[{"secondary_y": True}]])  # Secondary Y-axis for better clarity

    # Add Cure Rate histogram
    hist_fig.add_trace(
        go.Histogram(
            x=top_states['Cure Rate (%)'],
            name='Cure Rate (%)',
            marker_color='green',
            opacity=0.75
        ),
        secondary_y=False
    )

    # Add Death Rate histogram
    hist_fig.add_trace(
        go.Histogram(
            x=top_states['Death Rate (%)'],
            name='Death Rate (%)',
            marker_color='red',
            opacity=0.75
        ),
        secondary_y=False
    )

    # Customize histogram layout
    hist_fig.update_layout(
        title_text=f'Distribution of Cure and Death Rates in Top {top_n} States',
        barmode='overlay',  # Overlap bars for better comparison
        xaxis_title='Percentage (%)',
        yaxis_title='Frequency',
        plot_bgcolor='rgba(0,0,0,0)',
        legend_title='Rate Type'
    )

    # Show the plots
    bar_fig.show()
    pie_fig.show()
    hist_fig.show()

else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
